In [1]:
import numpy as np
from datetime import datetime
import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
from sympy import init_printing
init_printing()
def BlackandScholesPut(S, K, T, r, sigma):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    put = (K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0) - S * si.norm.cdf(-d1, 0.0, 1.0))
    
    return put
import pandas as pd
import plotly.express as px
import dash
import plotly.graph_objs as go
import pylab as pl
import matplotlib.pyplot as plt
import plotly.io as pio
#atualizar de acordo com a semana posição dos assessores em estruturas
DF=pd.read_excel('Posições AAI (3).xlsx')
DF['Financeiro']=DF['Size']*DF['Price TX']
série=str(input("A partir de qual série você deseja fazer o dataframe:"))
Série=[]
df=[]
for x in DF['Ticker']:
    Série.append(x[4:5])
DF['Série']=Série    
DF=DF.sort_values(by='Série')
DF=DF[DF['Série']>=f'{série}']
Acoes_geral=[]
for x in DF['Ticker']:
    Acoes_geral.append(x[0:4])
DF['Ticker Ação']=Acoes_geral
#Mudar ocasionalmente a base de opções em aberto na B3
Df=pd.read_excel('Pasta1.xlsx')
Df['Strike']=Df['Strike']/1000000000000
Df=Df.dropna()
d=Df['Ticker Ação'].apply(lambda x:x.replace(' ',''))
Df['Ticker Ação']=list(d)
f=Df['Ticker Opção'].apply(lambda x:x.replace(' ',''))
Df['Ticker Opção']=list(f)
opcoes=list(DF['Ticker'])
h=[]
for x in opcoes:
    
    Df[Df['Ticker Opção']==x]['Strike'].apply(lambda i:h.append(i)) 
DF['Strike']=h    

A partir de qual série você deseja fazer o dataframe:V


In [27]:
tp=[]
for x in opcoes:
    
    Df[Df['Ticker Opção']==x]['Tipo de Ação'].apply(lambda i:tp.append(i)) 
DF['Tipo de Ação']=tp
DF['Tipo de Ação']=DF['Tipo de Ação'].apply(lambda x:x.replace(' ',''))
num=[]

for x in DF.index:
    if DF['Tipo de Ação'][x][0:3]=='PNN':
        num.append('4')
    elif DF['Tipo de Ação'][x][0:3]=='PNA':
        num.append('5')
    elif DF['Tipo de Ação'][x][0:3]=='UNT':
        num.append('11')
    elif DF['Tipo de Ação'][x][0:2]=='ON':
        num.append('3')
    elif DF['Tipo de Ação'][x][0:2]=='CI':
        num.append('11')
    else:
        print(f'Opa Faltou essa {x}')
DF['TP Opção']=num        
for x in DF.index:
    DF['Ticker Ação'][x]=str(DF['Ticker'][x][0:4])+DF['TP Opção'][x]

<ipython-input-27-33fda55dcd6d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Ticker Ação'][x]=str(DF['Ticker'][x][0:4])+DF['TP Opção'][x]


In [42]:
vol_diaria=pd.read_excel('Voldiaria.xlsx')

In [43]:
f=list(dict.fromkeys(DF['Ticker Ação']))
z=list(vol_diaria['Ticker'])

In [44]:
list(set(f) - set(z))

In [45]:
vol_diaria=pd.read_excel('Voldiaria.xlsx')
vol=[]
for x in DF['Ticker Ação']:
    for y in vol_diaria.index:
        if x==vol_diaria['Ticker'][y]:
            vol.append(vol_diaria['Vol'][y])
DF['Volatilidade(%)']=vol 
vencimento=[]
for x in DF['Série']:
    if x=='U':
        vencimento.append('2021-09-17')
    elif x=='V':
        vencimento.append('2021-10-15')
    elif x=='W':
        vencimento.append('2021-11-19')
    elif x=='X':
        vencimento.append('2021-12-17')
    else:
        print('deu ruim')
DF['Data de Vencimento']=vencimento   
from workalendar.america import Brazil
diasuteis=[]
for x in DF['Data de Vencimento']:
    start_date =str(datetime.today())[0:10] 
    end_date = x

    start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
    end_datetime = datetime.strptime(end_date, '%Y-%m-%d')

    cal = Brazil()

    diasuteis.append(cal.get_working_days_delta(start_datetime, end_datetime))
DF['Dias Úteis']=diasuteis
Preço=[]
for x in DF['Ticker Ação']:
    for y in vol_diaria.index:
        if x==vol_diaria['Ticker'][y]:
            Preço.append(vol_diaria['Preço'][y])
DF['Preço Atual']=Preço
        

In [47]:
import yfinance
precos=[]
tic=[]
for x in list(dict.fromkeys(DF['Ticker Ação'])):
    T=yfinance.Ticker(f'{x}'+'.SA')
    Cotação=T.history(period='1d')
    Cotação['Close'].apply(lambda x:precos.append(x))
    tic.append(x)
Cotação=pd.DataFrame()
Cotação['Preço']=precos
Cotação['Ticker']=tic    

In [48]:
preçogeral=[]
for x in DF['Ticker Ação']:
    for y in Cotação.index:
        if x==Cotação['Ticker'][y]:
            preçogeral.append(Cotação['Preço'][y])

In [49]:
DF['Preço Atual']=preçogeral

In [54]:
Bs=[]
for x in BPS.index:
    maturidade=(BPS['Dias Úteis'][x]/252)
    vol_ajust=(BPS['Volatilidade(%)'][x]/100)
    juros=0.0625
    Bs.append(BlackandScholesPut(BPS['Preço Atual'][x],BPS['Strike'][x],maturidade,juros,vol_ajust))
BPS['Valor']=Bs    
d=BPS['Valor']/BPS['Price TX']
Situacao=[]
for x in d.index:
    if  d[x]<=0.85:
        Situacao.append('Vale a Pena Zerar')
    elif d[x]<=1.25 and d[x]>=0.85:
        Situacao.append('Vale ficar de olho')
    else:
        Situacao.append('Aguardar') 
        
BPS['Situação']=Situacao   

<ipython-input-54-90843dfd9e80>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BPS['Valor']=Bs
<ipython-input-54-90843dfd9e80>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BPS['Situação']=Situacao


In [71]:
idx=[]
for x in BPS['#']:
    if BPS[BPS['#']==x]['Operação'].count()==2:
        idx.append(x)                                                                                                 
        

In [73]:
for x in idx:
    dfbps=BPS[BPS['#']==x].sort_values(by='Strike',ascending=False)
    for y in dfbps.index:
        indices=[]
        indices.append(y)
        ind=indices[0]
        ind2=indices[1]
        valoratual=dfbps['Valor'][ind]-dfbps['Valor'][ind2]
        BPS['Valor'][ind]=valoratual
        BPS['Exposição Financeira'][ind]=(BPS['Strike'][ind]-BPS['Strike'][ind2])+(BPS['Price TX'][ind]-BPS['Price TX'][ind2]) *BPS['Size'][ind]
        BPS['Lucro ou Prejuízo (%)'][ind]=BPS['Valor'][ind]-(BPS['Price TX'][ind]-BPS['Price TX'][ind2])/(BPS['Size']*(BPS['Strike'][ind]-BPS['Strike'][ind2]))   

IndexError: list index out of range

In [51]:
BPS=DF[DF['Estrutura']=='Bull Put Spread']
DS=DF[DF['Estrutura']=='Venda de Put']

In [104]:
bps=[]
for x in DF.index:
    if DF['Estrutura'][x]=='Bull Put Spread':
        bps.append(DF['#'][x])

In [7]:
Bs=[]
for x in DF.index:
    maturidade=(DF['Dias Úteis'][x]/252)
    vol_ajust=(DF['Volatilidade(%)'][x]/100)
    juros=0.0525
    Bs.append(BlackandScholesPut(DF['Preço Atual'][x],DF['Strike'][x],maturidade,juros,vol_ajust))
DF['Valor']=Bs    
DF['Exposição Financeira']=DF['Size']*(DF['Preço Atual']-DF['Price TX'])
DF['Lucro ou Prejuízo (%)']=((DF['Price TX']-DF['Valor'])/DF['Strike'])*100
d=DF['Valor']/DF['Price TX']
Situacao=[]
for x in d.index:
    if  d[x]<=0.85:
        Situacao.append('Vale a Pena Zerar')
    elif d[x]<=1.25 and d[x]>=0.85:
        Situacao.append('Vale ficar de olho')
    else:
        Situacao.append('Aguardar') 
        
DF['Situação']=Situacao   

In [133]:
ndafazer=[]
indices=[]
for x in bps:
    if DF[DF['#']==x]['Operação'].count()==2:
        dfbps=DF[DF['#']==x].sort_values(by='Strike',ascending=False)
        indices=[]
        for y in dfbps.index:
            indices.append(y)
        ind=indices[0]
        ind2=indices[1]
        valoratual=dfbps['Valor'][ind]-dfbps['Valor'][ind2]
        DF['Valor'][ind]=valoratual
        ticker=DF['Ticker'][ind]
        DF['Ticker'][ind]=f'BPS {ticker}'
        DF['Exposição Financeira'][ind]=((DF['Strike'][ind]-DF['Strike'][ind2])-DF['Price TX'][ind])
        DF['Lucro ou Prejuízo (%)'][ind]=((DF['Price TX'][ind]-DF['Valor'][ind])/DF['Strike'][ind])*100
    else:
        ndafazer.append(x)
        
        
        

<ipython-input-133-c4c46a9d8e2a>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-133-c4c46a9d8e2a>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-133-c4c46a9d8e2a>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-133-c4c46a9d8e2a>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

In [113]:
DF['Strike'][ind]

In [134]:
d=DF.dropna()
d[d['Estrutura']=='Bull Put Spread']

,#,Operação,Status,Data,Ticker,Side,Size,Price TX,Estrutura,AAI,...,Ticker Ação,Strike,Volatilidade(%),Data de Vencimento,Dias Úteis,Preço Atual,Valor,Exposição Financeira,Lucro ou Prejuízo (%),Situação
513,1409,Swing Trade,Fechada,2021-08-10,BPS BPS SULA11 31.0-29.0,B,17000.0,0.50,Bull Put Spread,GABRIEL MALTA,...,SULA11,31.00,28.0,2021-10-15,13,27.370001,-3.403511,1.500000e+00,12.591972,Aguardar
509,1405,Swing Trade,Fechada,2021-08-05,VALEV91,S,50000.0,0.35,Bull Put Spread,GUILHERME,...,VALE3,104.91,32.5,2021-10-15,13,78.529999,26.096333,3.909000e+06,-24.541352,Aguardar
525,1421,Swing Trade,Fechada,2021-08-13,JBSSV345,B,75000.0,0.45,Bull Put Spread,VITOR (MESA PRIME),...,JBSS3,32.48,34.0,2021-10-15,13,35.599998,0.140301,2.636250e+06,0.953508,Vale a Pena Zerar
556,1457,Swing Trade,Aberta,2021-08-23,BPS BPS BPS BPAC11 29.86-29.86,B,10000.0,0.45,Bull Put Spread,VITOR (MESA PRIME),...,BPAC11,29.86,36.0,2021-10-15,13,26.920000,-14.900603,-4.500000e-01,51.408582,Aguardar
558,1457,Swing Trade,Aberta,2021-08-23,BPACV300,B,20000.0,0.45,Bull Put Spread,VITOR (MESA PRIME),...,BPAC11,29.86,36.0,2021-10-15,13,26.920000,2.980121,5.294000e+05,-8.473277,Aguardar
574,1475,Swing Trade,Aberta,2021-08-27,CYREV211,B,2000.0,0.40,Bull Put Spread,GABRIEL (MESA PRIME),...,CYRE3,21.16,49.0,2021-10-15,13,19.180000,2.165068,3.756000e+04,-8.341532,Aguardar
555,1459,Swing Trade,Aberta,2021-08-23,BPACV300,B,50000.0,0.45,Bull Put Spread,GUILHERME,...,BPAC11,29.86,36.0,2021-10-15,13,26.920000,2.980121,1.323500e+06,-8.473277,Aguardar
530,1432,Swing Trade,Aberta,2021-08-17,CYREV199,B,5000.0,0.50,Bull Put Spread,VITOR (MESA PRIME),...,CYRE3,19.91,49.0,2021-10-15,13,19.180000,1.245804,9.340000e+04,-3.745878,Aguardar
526,1422,Swing Trade,Aberta,2021-08-13,LRENV322,B,2000.0,0.40,Bull Put Spread,GABRIEL MALTA,...,LREN3,41.82,34.0,2021-10-15,13,36.639999,5.125196,7.248000e+04,-11.298891,Aguardar
494,1390,Swing Trade,Fechada,2021-07-29,PCARV330,B,8000.0,0.40,Bull Put Spread,GABRIEL MALTA,...,PCAR3,33.08,34.0,2021-10-15,13,26.590000,6.402310,2.095200e+05,-18.144832,Aguardar


In [17]:
assessor=['xxxxxxxxxx']

In [33]:
for x in assessor:
    
    df=p[p['AAI']==x]
    fig = px.scatter(df,
                     x="Volatilidade(%)", y="Lucro ou Prejuízo (%)", size="Exposição Financeira", color="Ticker Ação",
                     log_x=True, size_max=40,
                     template='plotly_dark', title=x,hover_name='Ticker',text='Ticker')
    fig.show()

    fig = px.sunburst(df[df['Situação']=='Vale a Pena Zerar'], path=['Situação','Ticker'], values='Exposição Financeira')
    fig.show()

KeyError: 'Ticker_path_copy'

In [31]:
tp=[]
for x in opcoes:
    
    Df[Df['Ticker Opção']==x]['Tipo de Ação'].apply(lambda i:tp.append(i)) 
DF['Tipo de Ação']=tp
DF['Tipo de Ação']=DF['Tipo de Ação'].apply(lambda x:x.replace(' ',''))

In [49]:
num=[]

for x in DF.index:
    if DF['Tipo de Ação'][x][0:3]=='PNN':
        num.append('3')
    elif DF['Tipo de Ação'][x][0:3]=='PNA':
        num.append('5')
    elif DF['Tipo de Ação'][x][0:3]=='UNT':
        num.append('11')
    elif DF['Tipo de Ação'][x][0:2]=='ON':
        num.append('3')
DF['TP Opção']=num        

In [59]:
for x in DF.index:
    DF['Ticker Ação'][x]=str(DF['Ticker'][x][0:4])+DF['TP Opção'][x]

<ipython-input-59-87c4293315b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Ticker Ação'][x]=str(DF['Ticker'][0:4])+DF['TP Opção'][x]
